In [ ]:
%env CUDA_LAUNCH_BLOCKING=1
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
%load_ext autoreload
%autoreload 2
from GraPL import hyperparameter_profiles, segment_bsds
import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def run_experiment(base_params, continuity_weights, num_trials=2):
    results_base_dir = f'experiment_results/continuity_ablation'
    aggregate_scores = {}
    for continuity_weight in continuity_weights:
        print(f'Running experiment with continuity_weight={continuity_weight}...')
        results_dir = f'{results_base_dir}/{continuity_weight}'
        trials = []
        mean_over_trials = {}
        with tqdm.tqdm(total=num_trials * 200) as progress_bar:
            for trial_num in range(num_trials):
                params = base_params.copy()
                params['continuity_weight'] = continuity_weight
                params['seed'] = trial_num
                trial_results_dir = f'{results_dir}/{trial_num}'
                trial_scores = segment_bsds(results_dir=trial_results_dir, resume=True, progress_bar=progress_bar, **params)
                image_ids = list(trial_scores.keys())
                metrics = trial_scores[image_ids[0]].keys()
                trial_scores = {metric: np.mean([trial_scores[id][metric] for id in image_ids]) for metric in metrics}
                trials.append(trial_scores)
        for metric in trials[0]:
            mean_over_trials[metric] = np.mean([trial[metric] for trial in trials])
        aggregate_scores[continuity_weight] = mean_over_trials
        print(f'Scores for continuity_weight={continuity_weight}: {mean_over_trials["accuracy"]}')
    return aggregate_scores

base_params = hyperparameter_profiles["best_miou"]

continuity_weights = [0, 0.25, 0.5, 0.75, 1, 1.5, 2, 3, 4, 6, 8]  
continuity_ranges = [1]

run_experiment(base_params, continuity_weights, num_trials=10)